In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import transformer
import gpytorch
import botorch 

# BO for Minimizing Emittance*Bmag with 9 Variables (SQ, CQ, SOL, matching quads)

In [2]:
#load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-14 11:27:54.632738: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Import design Twiss parameters (OTR2)

In [3]:
beamline_info = json.load(open('../configs/beamline_info.json'))
get_twiss0 = beamline_info['Twiss0']
# emit, beta, alpha
twiss0 = {'x': [get_twiss0[0], get_twiss0[2], get_twiss0[4]],
          'y': [get_twiss0[1], get_twiss0[3], get_twiss0[5]]}

beta0_x, alpha0_x = twiss0['x'][1], twiss0['x'][2]
beta0_y, alpha0_y = twiss0['y'][1], twiss0['y'][2]
print(twiss0['x'])
print(twiss0['y'])

[1e-06, 1.113081026, -0.0689403587]
[1e-06, 1.113021659, -0.07029489754]


## Objective Function

In [4]:
# convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

# input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','CQ01:b1_gradient', 'SQ01:b1_gradient',
                 "QA01:b1_gradient", "QA02:b1_gradient", 
                 "QE01:b1_gradient", "QE02:b1_gradient", "QE03:b1_gradient", "QE04:b1_gradient"]

# output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx, vary, varz, var1, var2, var3, var4, var5, var6): 
    """
    vars: sol, cq, sq, qa1, qa2, q1, q2, q3, q4
    """ 
    # make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    # fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    # set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz
    x_in[:, Model.loc_in[opt_var_names[3]]] = var1
    x_in[:, Model.loc_in[opt_var_names[4]]] = var2
    x_in[:, Model.loc_in[opt_var_names[5]]] = var3
    x_in[:, Model.loc_in[opt_var_names[6]]] = var4
    x_in[:, Model.loc_in[opt_var_names[7]]] = var5
    x_in[:, Model.loc_in[opt_var_names[8]]] = var6

    # output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]



def objective(y_out):
    
    # output is geometric emittance in transverse plane
    out1 = y_out[:, Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:, Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    emit = np.sqrt(out1 * out2)
  
    sigma_x = y_out[:, Model.loc_out['sigma_x']] #grab sigma_x out of the model 
    sigma_y = y_out[:, Model.loc_out['sigma_y']] #grab sigma_y out of the model 
    
    # real beta and alpha 
    # NEEDS TO BE FIXED - currently assuming real alpha to be the same as design alpha 
    alpha_x, alpha_y = alpha0_x, alpha0_y
    beta_x, beta_y = (sigma_x**2) / out1, (sigma_y**2) / out2
    
    # bmag 
    bmag_x = 0.5 * ((beta0_x / beta_x) + (beta_x / beta0_x)) + 0.5 * (alpha_x * np.sqrt(beta0_x / beta_x) - alpha0_x * np.sqrt(beta_x / beta0_x))
    bmag_y = 0.5 * ((beta0_y / beta_y) + (beta_y / beta0_y)) + 0.5 * (alpha_y * np.sqrt(beta0_y / beta_y) - alpha0_y * np.sqrt(beta_y / beta0_y))
    bmag = np.sqrt(bmag_x * bmag_y)
    print(f'bmag: {bmag} emit: {emit}') 
    
    return (emit * bmag)/1e-6
    #return np.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [5]:
def get_BO_point(x, f, bounds, beta=2.5, precision = None, phys=True):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    if phys == True:
        gp = physics_gp.PhysicsExactGPModel(x, f.flatten(), likelihood, precision)
    else:
        gp = botorch.models.SingleTaskGP(x, f, likelihood)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    #print('training hyperparameters')
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    #print('optimizing acquisition function')
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,#5
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [6]:
#specify bounds
bounds = {'varx': (0.46, 0.485),
          'vary': (-0.02, 0.02),
          'varz': (-0.02, 0.02),
          'var1': (-4, -1),
          'var2': (1, 4),
          'var3': (-7, -1),
          'var4': (-1, 7),
          'var5': (-1, 7),
          'var6': (-7, 1)}
                        
tbounds = torch.tensor([[bounds['varx'][0], bounds['varx'][1]],
                        [bounds['vary'][0], bounds['vary'][1]], 
                        [bounds['varz'][0], bounds['varz'][1]],
                        [bounds['var1'][0], bounds['var1'][1]],
                        [bounds['var2'][0], bounds['var2'][1]],
                        [bounds['var3'][0], bounds['var3'][1]],
                        [bounds['var4'][0], bounds['var4'][1]],
                        [bounds['var5'][0], bounds['var5'][1]],
                        [bounds['var6'][0], bounds['var6'][1]]])

#print(bounds['varx'][0])
#print(tbounds)

#create initial samples within specified bounds
n_samples = 5
n_var = 9
train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(tbounds[i,0],tbounds[i,1],(n_samples,)))
#train_x = torch.as_tensor(train_x).type(torch.FloatTensor)

"""
train_x = torch.rand(n_samples,3) 
train_x[:,0] = train_x[:,0] * diff['varx'] + bounds['varx'][0]
train_x[:,1] = train_x[:,1] * diff['vary'] + bounds['vary'][0]
train_x[:,2] = train_x[:,2] * diff['varz'] + bounds['varz'][0]
""" 
print(train_x)

train_y = torch.tensor([evaluate(vars[0], vars[1], vars[2], vars[3], vars[4], vars[5], vars[6], vars[7], vars[8]) for vars in train_x]).reshape(-1,1)
print(train_y)


#transformer 
transformer_x = transformer.Transformer(tbounds.transpose(0,1), transform_type = 'normalize')
print(transformer_x.mins)
print(transformer_x.maxs)
normed_train_x = transformer_x.forward(train_x)

transformer_y = transformer.Transformer(train_y, transform_type = 'standardize') 
normed_train_y = transformer_y.forward(train_y)

""" 
#create initial model
gp = SingleTaskGP(normed_train_x, normed_train_y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);
"""

tensor([[ 4.6418e-01, -1.9117e-03, -1.3368e-03, -1.9305e+00,  3.9036e+00,
         -3.0059e+00,  8.2373e-01,  5.7331e+00, -2.4869e+00],
        [ 4.6736e-01, -1.5112e-02, -1.5957e-03, -1.7717e+00,  2.4311e+00,
         -4.4188e+00,  1.8396e+00,  8.5522e-01, -2.7316e+00],
        [ 4.6643e-01,  1.8102e-02,  1.1459e-03, -2.6752e+00,  1.7924e+00,
         -4.5148e+00,  6.9346e+00, -8.0024e-01,  2.8992e-01],
        [ 4.6132e-01,  3.5913e-03, -1.2952e-02, -3.6826e+00,  3.7173e+00,
         -4.7937e+00,  1.2303e+00,  3.9029e+00, -1.0695e+00],
        [ 4.7832e-01, -6.9391e-03, -4.8378e-03, -1.1993e+00,  3.7667e+00,
         -6.5673e+00,  5.3081e+00,  5.5752e+00, -5.1483e+00]])


2022-06-14 11:28:02.590471: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-14 11:28:02.615443: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996075000 Hz


bmag: [1.5233376] emit: [1.069796e-06]
bmag: [1.6119298] emit: [1.081863e-06]
bmag: [2.5465887] emit: [1.4116972e-06]
bmag: [3.7971401] emit: [1.1632793e-06]
bmag: [1.0842239] emit: [8.529898e-07]
tensor([[-1.6297],
        [-1.7439],
        [-3.5950],
        [-4.4171],
        [-0.9248]])
[ 0.46 -0.02 -0.02 -4.    1.   -7.   -1.   -1.   -7.  ]
[ 0.485  0.02   0.02  -1.     4.    -1.     7.     7.     1.   ]


' \n#create initial model\ngp = SingleTaskGP(normed_train_x, normed_train_y)\nmll = ExactMarginalLogLikelihood(gp.likelihood, gp)\nfit_gpytorch_model(mll);\n'

## Bayesian Optimization

In [7]:
n_steps = 20
best_y = torch.max(train_y)
for i in range(n_steps):
    if i > 10:
        phys_l = True
        
    bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)
    #x_new, model = get_BO_point(train_x, normed_train_y, bounds=tbounds.transpose(0,1), phys = False)
    x_new, model = get_BO_point(normed_train_x, normed_train_y, bounds=bounds, phys = False)
    
    #train_x = torch.cat((train_x, x_new))
    train_x = torch.cat((train_x, transformer_x.backward(x_new)))
    normed_train_x = transformer_x.forward(train_x)
    
    new_y = torch.tensor(evaluate(train_x[-1,0], train_x[-1,1], train_x[-1,2], 
                                 train_x[-1,3], train_x[-1,4], train_x[-1,5],
                                 train_x[-1,6], train_x[-1,7], train_x[-1,8])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    normed_train_y = transformer_y.forward(train_y)
    
    train_x = train_x.type(torch.FloatTensor)
    train_y = train_y.type(torch.FloatTensor)
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iter     target      varx       vary      varz      var1      var2      var3      var4      var5      var6")
    print(f'{color[0]}{i+1}      {train_y[-1,0]:.5f}   {train_x[-1,0]:.5f}   {train_x[-1,1]:.5f}   {train_x[-1,2]:.5f}   {train_x[-1,3]:.5f}   {train_x[-1,4]:.5f}   {train_x[-1,5]:.5f}   {train_x[-1,6]:.5f}   {train_x[-1,7]:.5f}   {train_x[-1,8]:.5f}{color[1]}')
    #print(f'max = {torch.max(train_y):.5f}')

bmag: [1.913289] emit: [7.68876e-07]
iter     target      varx       vary      varz      var1      var2      var3      var4      var5      var6
1      -1.47108   0.47377   -0.00776   -0.00276   -1.31384   3.60709   -5.33280   3.73193   5.02076   -4.32158
bmag: [1.9754714] emit: [8.7927197e-07]
iter     target      varx       vary      varz      var1      var2      var3      var4      var5      var6
2      -1.73698   0.47861   -0.01221   -0.00529   -1.22854   3.23794   -6.96081   5.31290   3.74318   -5.11190
bmag: [1.1159384] emit: [1.1442831e-06]
iter     target      varx       vary      varz      var1      var2      var3      var4      var5      var6
3      -1.27695   0.47844   -0.00741   -0.00912   -1.35550   4.00000   -6.30907   3.79619   6.82383   -6.09028
bmag: [2.602773] emit: [1.1662045e-06]
iter     target      varx       vary      varz      var1      var2      var3      var4      var5      var6
4      -3.03537   0.48010   -0.00522   -0.00075   -1.25901   4.00000   -6.79833   4

In [8]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""

'\n#define acquisition function\nfrom botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement\nfrom botorch.optim import optimize_acqf\n\n#optimize\nn_steps = 45\nfor i in range(n_steps):\n    #best_normed_y = torch.max(normed_train_y)\n    UCB = UpperConfidenceBound(gp, beta=2.5)\n    #EI = ExpectedImprovement(gp, best_normed_y)\n\n    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)\n    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)\n\n    train_x = torch.cat((train_x, transformer_x.backward(candidate)))\n    normed_train_x = transformer_x.forward(train_x)\n\n    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)\n    train_y = torch.cat((train_y, new_y))\n    \n    print("iteration        target         varx         vary         varz")\n    print(f\'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-